In [666]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

matplotlib.style.use('ggplot')
%matplotlib inline

In [661]:
df = pd.read_csv("train_knn.csv", sep=",")
df_test = pd.read_csv("test_knn.csv", sep=",")

print(len(df))

df['titleType'] = df['titleType'].astype('category')
df_test['titleType'] = df_test['titleType'].astype('category')

df.head()

2000


,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,mainDirector,numVotes,mainDirectorBirthYear,...,MainWriterPrimaryProfession,titleLength,genres,genderVoters,worldPromotion,Downloads,prefVotersCountry,firstShownCountry,Cash,rating
0,short,Susans längtan,Susans längtan,0,2009,NaN,23.0,Kristian A. Söderström,32,NaN,...,"producer,director,writer",14,"['Drama', 'Horror', 'Short']",17.0,1,7658,SM,KM,34010,6
1,movie,Evangelion: 2.0 You Can (Not) Advance,Evangerion shin gekijôban: Ha,0,2009,NaN,112.0,Masayuki,13110,1961.0,...,"director,art_department,writer",29,"['Action', 'Animation', 'Drama']",6513.0,0,2703,LR,MV,21356,8
2,short,The Grocer,The Grocer,0,2010,NaN,7.0,Ivan Miguel Guerrero,5,NaN,...,"producer,writer",10,"['Crime', 'Drama', 'Short']",2.0,1,7541,TC,CM,27177,9
3,tvMovie,La part de l'ombre,La part de l'ombre,0,2000,NaN,NaN,Philippe Venault,6,1947.0,...,writer,18,"['Drama', 'Thriller', None]",1.0,1,4446,SZ,BQ,55620,6
4,movie,Aalaapana,Aalaapana,0,1986,NaN,NaN,Vamshi,10,1956.0,...,"actor,writer,director",9,"['Drama', 'Musical', 'Thriller']",7.0,0,4671,BM,LT,5147,7


In [662]:
X_train = df[['numVotes', 'worldPromotion', 'genres', 'mainDirectorBirthYear', 'mainWriterBirthYear', 'Downloads', 'Cash']]
y_train = df["rating"]

X_test = df_test[['numVotes', 'worldPromotion', 'genres', 'mainDirectorBirthYear', 'mainWriterBirthYear', 'Downloads', 'Cash']]

g = ['Music', 'Mystery', 'Comedy', 'Adventure', 'History', 'War', 'Reality-TV', 'News', 'Sport', 'Drama', 'Game-Show', 'Romance', 'Family', 'Thriller', 'Documentary', 'Biography', 'Adult', 'Horror', 'Animation', 'Western', 'Talk-Show', 'Short', 'Crime', 'Musical', 'Sci-Fi', 'Fantasy', 'Action', 'Film-Noir']
for v in g:
    X_train[v] = [0]*2000
    X_test[v] = [0]*2000
i = 0
for genres in X_train['genres']:
    try:
        arr = genres[1:-1].replace("'", "").split(', ')
        for v in arr:
            if v != 'nan' and v != 'None': X_train[v][i] = 1
    except:
        print(genres)
    i+=1

i = 0
for genres in X_test['genres']:
    try:
        arr = genres[1:-1].replace("'", "").split(', ')
        for v in arr:
            if v != 'nan' and v != 'None': X_test[v][i] = 1
    except:
        print(genres)
    i+=1
    
X_train.pop('genres')
X_test.pop('genres')
    

#X_train = pd.get_dummies(X_train, columns=['titleType'], drop_first=True)
#X_test = pd.get_dummies(X_test, columns=['titleType'], drop_first=True)

<ipython-input-662-9c9a68a7e9c1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[v] = [0]*2000
<ipython-input-662-9c9a68a7e9c1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[v] = [0]*2000
<ipython-input-662-9c9a68a7e9c1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if v != 'nan' and v != 'None': X_tr

0                  ['Music', None, None]
1             ['Drama', 'History', None]
2          ['Crime', 'Drama', 'Mystery']
3                  ['Adult', None, None]
4        ['Action', 'Comedy', 'Romance']
                      ...               
1995    ['Animation', 'Comedy', 'Crime']
1996    ['Animation', 'Family', 'Short']
1997              ['Comedy', None, None]
1998            ['Drama', 'Music', None]
1999      ['Action', 'Comedy', 'Family']
Name: genres, Length: 2000, dtype: object

In [663]:
X_train.head()

,numVotes,worldPromotion,mainDirectorBirthYear,mainWriterBirthYear,Downloads,Cash,Music,Mystery,Comedy,Adventure,...,Animation,Western,Talk-Show,Short,Crime,Musical,Sci-Fi,Fantasy,Action,Film-Noir
0,32,1,NaN,NaN,7658,34010,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,13110,0,1961.0,1960.0,2703,21356,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5,1,NaN,NaN,7541,27177,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
3,6,1,1947.0,NaN,4446,55620,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,0,1956.0,NaN,4671,5147,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [664]:
X_test.head()

,numVotes,worldPromotion,mainDirectorBirthYear,mainWriterBirthYear,Downloads,Cash,Music,Mystery,Comedy,Adventure,...,Animation,Western,Talk-Show,Short,Crime,Musical,Sci-Fi,Fantasy,Action,Film-Noir
0,8,0,1942.0,1885.0,9453,16994,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,62,0,1939.0,1948.0,7380,58527,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,89,1,NaN,1944.0,2240,47404,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,30,0,1936.0,NaN,1699,5693,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,623,1,1947.0,1962.0,5291,45958,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [665]:
def create_pipeline():
    return Pipeline([
        ("fill_nan", SimpleImputer(missing_values=np.nan, strategy='mean')), 
        ("scale", StandardScaler()),
        ("clf", KNeighborsClassifier(n_neighbors=10))
    ])

pipline = create_pipeline()

pipline.fit(X_train, y_train)

y_pred = pipline.predict(X_test)

subm = pd.DataFrame({'Id': range(X_test.shape[0]), 'class': y_pred})
subm.to_csv('submission.csv', index=False)

In [659]:
for i in range(0, 20):
    print(y_test[i], y_pred[i])

5 7
6 6
7 7
8 7
6 8
4 7
9 8
6 8
10 6
1 8
1 8
9 8
5 8
6 8
1 7
3 8
3 7
7 7
8 7
1 8


In [ ]:
g = ['Music', 'Mystery', 'Comedy', 'Adventure', 'History', 'War', 'Reality-TV', 'News', 'Sport', 'Drama', 'Game-Show', 'Romance', 'Family', 'Thriller', 'Documentary', 'Biography', 'Adult', 'Horror', 'Animation', 'Western', 'Talk-Show', 'Short', 'Crime', 'Musical', 'Sci-Fi', 'Fantasy', 'Action', 'Film-Noir']
for v in g:
    X_train[v] = [0]*2000
    X_test[v] = [0]*2000
i = 0
for genres in X_train['genres']:
    try:
        arr = genres[1:-1].replace("'", "").split(', ')
        for v in arr:
            if v != 'nan' and v != 'None': X_train[v][i] = 1
    except:
        print(genres)
    i+=1

i = 0
for genres in X_test['genres']:
    try:
        arr = genres[1:-1].replace("'", "").split(', ')
        for v in arr:
            if v != 'nan' and v != 'None': X_test[v][i] = 1
    except:
        print(genres)
    i+=1
    
X_train.pop('genres')
X_test.pop('genres')
    